# Case 2: How LDES reacts to different reserve requirements?

## Scheme 1 (c1s2b20 - No reserve requirements)

In [1]:
using CSV, DataFrames, JuMP, HiGHS, SCIP, Clp

storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("cleaned","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("cleaned","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("cleaned","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("cleaned","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[1:1440,:];

In [2]:
function case2scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case2scheme1 (generic function with 1 method)

In [3]:
c2s1 = case2scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       6900     4.4070916831e+04 Pr: 18515(1.42102e+11); Du: 0(6.80984e-07) 5s
      11434     1.5345978785e+05 Pr: 29160(1.74669e+12); Du: 0(5.329e-07) 10s
      15028     2.2304843530e+05 Pr: 28469(1.56283e+12); Du: 0(7.36232e-07) 15s
      18398     2.7494359675e+05 Pr: 28988(5.20024e+10); Du: 0(8.2587e-07) 21s
      21739     3.1063229559e+05 Pr: 34237(1.07484e+12); Du: 0(9.90962e-07) 26s
      24753     3.5265079906e+05 Pr: 30738(1.07262e+11); Du: 0(1.33407e-06) 32s
      27376     3.8865359398e+05 Pr: 30926(7.1398e+11); Du: 0(1.07194e-06) 37s
      30063 

(CAPG = [4357.88057476665, 24147.563702701395], CAPS = [-0.0, 12915.264351236765, -0.0, -0.0, 2738.291371295227, 0.0, 0.0, 0.0, -0.0, -0.0, 5660.399999999995, -0.0, -0.0, -0.0], SOCM = [-0.0, 2.268530961275762e6, -0.0, -0.0, 199584.2600355829, -0.0, -0.0, -0.0, -0.0, -0.0, 66998.18181818399, 0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … 0.0 -0.0; 1.134265480637881e6 1.1429187077532096e6 … 1.1335166886378808e6 1.134265480637881e6; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 12915.264351236765 … 202.59999999978035 1117.6000000001973; … ; -0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -12915.264351236765 … -202.59999999978035 -1117.6000000001973; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 9314.10862870478 1414.2442774680112 … 0.0 0.

## Scheme 2 Fixed reserve requirements (1800MW)

In [4]:
function case2scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        R_DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] + R_DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))
    @constraint(model, c23[s in S, t in T], R_DISCHARGE[s,t] <= SOC[s,t])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) + sum(R_DISCHARGE[s,t] for s in S) >= 1800)

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        R_DISCHARGE = value.(R_DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case2scheme2 (generic function with 1 method)

In [5]:
c2s2 = case2scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
112275 rows, 91196 cols, 358940 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
Presolve : Reductions: rows 96555(-20096); columns 75476(-16714); elements 426981(+62643)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2880(4.33159e+08) 0s
       5278     1.8905553197e+04 Pr: 14846(1.97339e+11); Du: 0(8.80397e-07) 5s
       9273     2.8951803566e+04 Pr: 20990(2.47206e+11); Du: 0(1.96553e-06) 10s
      12832     3.8317800603e+04 Pr: 18709(2.54341e+11); Du: 0(2.21615e-06) 15s
      16039     5.0931427687e+04 Pr: 19946(1.05124e+12); Du: 0(2.09534e-06) 20s
      18920     8.0208237075e+04 Pr: 21631(1.66097e+12); Du: 0(2.42369e-06) 26s
      21882     1.4100931852e+05 Pr: 20134(3.22474e+12); Du: 0(1.27411e-06) 31s
      24766     1.9662393052e+05 Pr: 18817(

(CAPG = [5296.513304298667, 24291.5229724115], CAPS = [0.0, 11792.004177428, 0.0, 0.0, 2842.6637231578643, 0.0, 0.0, 0.0, -0.0, -0.0, 7596.695822703953, 0.0, -0.0, 0.0], SOCM = [-0.0, 2.0919121648686265e6, -0.0, -0.0, 221653.52406298718, -0.0, -0.0, -0.0, -0.0, -0.0, 67839.92404916289, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; 1.0459560824343133e6 1.05385672523319e6 … 1.0452072904343134e6 1.0459560824343133e6; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 -0.0 … 0.0 0.0; 0.0 11792.004177428 … 202.59999999999854 1117.5999999999985; … ; -0.0 -0.0 … 0.0 0.0; -0.0 0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], R_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 2000.0 … 0.0 2000.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 -11792.004177428 … -202.59999999999854 -1117.5999999999

## Scheme 3 Dynamic (Demand-based) Requirements (3% of Demand)

In [6]:
function case2scheme3(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        R_DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] + R_DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))
    @constraint(model, c23[s in S, t in T], R_DISCHARGE[s,t] <= SOC[s,t])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) + sum(R_DISCHARGE[s,t] for s in S) >= 0.03 * halfhourlydemand[halfhourlydemand.hh.==t,:load][1])

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        R_DISCHARGE = value.(R_DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case2scheme3 (generic function with 1 method)

In [7]:
c2s3 = case2scheme3(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
112275 rows, 91196 cols, 358940 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
Presolve : Reductions: rows 96555(-20096); columns 75476(-16714); elements 426981(+62643)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2880(4.14234e+08) 0s
       5278     1.8905553197e+04 Pr: 14846(1.9732e+11); Du: 0(8.80397e-07) 5s
       9273     2.8951803566e+04 Pr: 20990(2.47187e+11); Du: 0(1.96553e-06) 10s
      12832     3.8317800603e+04 Pr: 18709(2.54322e+11); Du: 0(2.21615e-06) 15s
      16264     5.2763462139e+04 Pr: 22149(1.26987e+12); Du: 0(2.3958e-06) 21s
      19177     8.4358948286e+04 Pr: 24223(1.88845e+12); Du: 0(2.4307e-06) 26s
      22451     1.5456579108e+05 Pr: 21507(9.18631e+11); Du: 0(1.42193e-06) 31s
      25262     2.0167507529e+05 Pr: 25830(1.0

(CAPG = [4772.413377074108, 24315.03754463519], CAPS = [-0.0, 11568.562455364809, 0.0, 0.0, 3610.5872174195356, 0.0, 0.0, 0.0, -0.0, -0.0, 7167.789405506708, 1.4784481517117632e-11, -0.0, 0.0], SOCM = [-0.0, 2.064518796567969e6, -0.0, -0.0, 279511.1777494713, -0.0, -0.0, -0.0, -0.0, -0.0, 66640.26737283451, 4.928160505705888e-10, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; 1.0322593982839845e6 1.040010335129079e6 … 1.0315106062839845e6 1.0322593982839845e6; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 -0.0 … 0.0 0.0; 0.0 11568.562455364809 … 202.59999999995412 1117.6000000000236; … ; -0.0 -0.0 … 0.0 0.0; -0.0 0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], R_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 865.3281521863225 … 0.0 992.61; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 -0.0; 0.0 -11

## Scheme 4 Dynamic (VRE-based) Requirements (10% Wind 4% Solar)

In [13]:

function case2scheme4(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        R_DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] + R_DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))
    @constraint(model, c23[s in S, t in T], R_DISCHARGE[s,t] <= SOC[s,t])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) + sum(R_DISCHARGE[s,t] for s in S) >= 0.04 * halfhourlyvrecf[halfhourlyvrecf.vre_id.==1,:cf][1] * vreinfo[vreinfo.name.=="Solar",:installedMW][1] + 0.1 * (halfhourlyvrecf[halfhourlyvrecf.vre_id.==2,:cf][1] * vreinfo[vreinfo.name.=="Onshore",:installedMW][1] + halfhourlyvrecf[halfhourlyvrecf.vre_id.==3,:cf][1] * vreinfo[vreinfo.name.=="Offshore",:installedMW][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        R_DISCHARGE = value.(R_DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case2scheme4 (generic function with 1 method)

In [14]:
c2s4 = case2scheme4(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
112275 rows, 91196 cols, 358940 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
Presolve : Reductions: rows 96555(-20096); columns 75476(-16714); elements 426981(+62643)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2880(4.95965e+08) 0s
       5068     1.8386828484e+04 Pr: 21663(1.42213e+11); Du: 0(6.49009e-07) 5s
       8835     2.7826494666e+04 Pr: 25763(2.59999e+11); Du: 0(1.11756e-06) 10s
      12372     3.6949055654e+04 Pr: 20499(2.20867e+11); Du: 0(1.49382e-06) 15s
      15810     4.9286633988e+04 Pr: 24112(7.79712e+11); Du: 0(1.88355e-06) 21s
      18671     7.6100660595e+04 Pr: 22919(1.23672e+12); Du: 0(2.37892e-06) 26s
      21558     1.3257487221e+05 Pr: 18327(2.28363e+12); Du: 0(1.85844e-06) 31s
      24512     1.9374252902e+05 Pr: 24735(

(CAPG = [7391.7094772132905, 24493.612468650303], CAPS = [-0.0, 11329.47260806384, -0.0, 0.0, 1258.4342289008505, 0.0, 0.0, 0.0, -0.0, -0.0, 10066.451217171714, -0.0, -0.0, 0.0], SOCM = [-0.0, 1.9825120755905863e6, -0.0, -0.0, 101641.7460268095, -0.0, -0.0, -0.0, -0.0, -0.0, 66269.26355371898, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; 991256.0377952931 998846.7844426959 … 990507.2457952931 991256.0377952931; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 -0.0 … 0.0 0.0; 0.0 11329.47260806384 … 202.59999999999854 1117.5999999999985; … ; -0.0 -0.0 … 0.0 0.0; -0.0 0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], R_DISCHARGE = [0.0 0.0 … 0.0 0.0; 3461.84577109915 4720.280000000001 … 0.0 4720.280000000001; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 -0.0; 0.0 -11329.47260806384 … -

## Scheme 5 ALL

In [15]:

function case2scheme5(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        R_DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] + R_DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))
    @constraint(model, c23[s in S, t in T], R_DISCHARGE[s,t] <= SOC[s,t])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) + sum(R_DISCHARGE[s,t] for s in S) >= 1800 + 0.03 * halfhourlydemand[halfhourlydemand.hh.==t,:load][1] + 0.04 * halfhourlyvrecf[halfhourlyvrecf.vre_id.==1,:cf][1] * vreinfo[vreinfo.name.=="Solar",:installedMW][1] + 0.1 * (halfhourlyvrecf[halfhourlyvrecf.vre_id.==2,:cf][1] * vreinfo[vreinfo.name.=="Onshore",:installedMW][1] + halfhourlyvrecf[halfhourlyvrecf.vre_id.==3,:cf][1] * vreinfo[vreinfo.name.=="Offshore",:installedMW][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        R_DISCHARGE = value.(R_DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case2scheme5 (generic function with 1 method)

In [16]:
c2s5 = case2scheme5(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
112275 rows, 91196 cols, 358940 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
96555 rows, 75476 cols, 426981 nonzeros
Presolve : Reductions: rows 96555(-20096); columns 75476(-16714); elements 426981(+62643)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2880(5.64775e+08) 0s
       5068     1.8386828484e+04 Pr: 21663(1.42282e+11); Du: 0(6.49009e-07) 5s
       9036     2.8411109664e+04 Pr: 21070(1.25427e+11); Du: 0(1.42187e-06) 10s
      12597     3.7675262536e+04 Pr: 18785(4.12701e+11); Du: 0(1.75539e-06) 15s
      16039     5.0931427687e+04 Pr: 19946(1.05137e+12); Du: 0(2.09534e-06) 21s
      18920     8.0208237075e+04 Pr: 21631(1.6611e+12); Du: 0(2.42369e-06) 26s
      22169     1.4578709031e+05 Pr: 24478(1.63647e+13); Du: 0(1.32153e-06) 31s
      25006     1.9909519030e+05 Pr: 17771(7

(CAPG = [9761.625401425474, 24319.28141103725], CAPS = [-0.0, 10006.906760473501, -0.0, 0.0, 1239.2039166284671, 0.0, 0.0, 0.0, -0.0, -0.0, 12627.652510435304, -0.0, -0.0, 0.0], SOCM = [-0.0, 1.7290342799912512e6, -0.0, -0.0, 100910.8788706051, -0.0, -0.0, -0.0, -0.0, -0.0, 55583.265625304935, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; 864517.1399956256 871221.7675251429 … 863768.3479956256 864517.1399956256; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 -0.0 … 0.0 0.0; 0.0 10006.906760473501 … 202.59999999999854 1117.5999999999985; … ; -0.0 -0.0 … 0.0 0.0; -0.0 0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], R_DISCHARGE = [0.0 0.0 … 0.0 0.0; 6165.776083371533 1602.6406771019656 … 0.0 7512.890000000001; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 -0.0; 0.0 -10006.906760473501